In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col,lit
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

spark

df = spark.read.option("header", "true")\
          .csv("/home/iceberg/data/events.csv")\
          .withColumn("event_date", expr("DATE_TRUNC('day', event_time)"))



# df.show()
df.join(df, lit(1) == lit(1)).take(5)

24/12/10 10:31:33 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
24/12/10 10:31:36 WARN Column: Constructing trivially true equals predicate, '1 = 1'. Perhaps you need to use aliases.


[Row(user_id='1037710827', device_id='532630305', referrer=None, host='www.zachwilson.tech', url='/', event_time='2021-03-08 17:27:24.241000', event_date=datetime.datetime(2021, 3, 8, 0, 0), user_id='1037710827', device_id='532630305', referrer=None, host='www.zachwilson.tech', url='/', event_time='2021-03-08 17:27:24.241000', event_date=datetime.datetime(2021, 3, 8, 0, 0)),
 Row(user_id='1037710827', device_id='532630305', referrer=None, host='www.zachwilson.tech', url='/', event_time='2021-03-08 17:27:24.241000', event_date=datetime.datetime(2021, 3, 8, 0, 0), user_id='925588856', device_id='532630305', referrer=None, host='www.eczachly.com', url='/', event_time='2021-05-10 11:26:21.247000', event_date=datetime.datetime(2021, 5, 10, 0, 0)),
 Row(user_id='1037710827', device_id='532630305', referrer=None, host='www.zachwilson.tech', url='/', event_time='2021-03-08 17:27:24.241000', event_date=datetime.datetime(2021, 3, 8, 0, 0), user_id='-1180485268', device_id='532630305', referrer=N

In [3]:
#local partition sort
sorted = df.repartition(10, col("event_date")) \
        .sortWithinPartitions(col("event_date"), col("host"), col("browser_family")) \
        .withColumn("event_time", col("event_time").cast("timestamp")) 

#global sort 
sortedTwo = df.repartition(10, col("event_date")) \
        .sort(col("event_date"), col("host"), col("browser_family")) \
        .withColumn("event_time", col("event_time").cast("timestamp")) 

sorted.explain
sortedTwo.explain

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `browser_family` cannot be resolved. Did you mean one of the following? [`user_id`, `referrer`, `device_id`, `event_date`, `event_time`].;
'Sort [event_date#29 ASC NULLS FIRST, host#20 ASC NULLS FIRST, 'browser_family ASC NULLS FIRST], false
+- RepartitionByExpression [event_date#29], 10
   +- Project [user_id#17, device_id#18, referrer#19, host#20, url#21, event_time#22, date_trunc(day, cast(event_time#22 as timestamp), Some(Etc/UTC)) AS event_date#29]
      +- Relation [user_id#17,device_id#18,referrer#19,host#20,url#21,event_time#22] csv


In [9]:
%%sql

CREATE DATABASE IF NOT EXISTS bootcamp

++
||
++
++

In [55]:
%%sql

DROP TABLE IF EXISTS bootcamp.events

++
||
++
++

In [56]:
%%sql

CREATE TABLE IF NOT EXISTS bootcamp.events (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
PARTITIONED BY (years(event_date));


++
||
++
++

In [112]:
%%sql


CREATE TABLE IF NOT EXISTS bootcamp.events_sorted (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg;

++
||
++
++

In [113]:
%%sql


CREATE TABLE IF NOT EXISTS bootcamp.events_unsorted (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg;

++
||
++
++

In [120]:

start_df = df.repartition(4, col("event_date")).withColumn("event_time", col("event_time").cast("timestamp")) \
    

first_sort_df = start_df.sortWithinPartitions(col("event_date"), col("browser_family"), col("host"))

sorted = df.repartition(10, col("event_date")) \
        .sortWithinPartitions(col("event_date")) \
        .withColumn("event_time", col("event_time").cast("timestamp")) \

start_df.write.mode("overwrite").saveAsTable("bootcamp.events_unsorted")
first_sort_df.write.mode("overwrite").saveAsTable("bootcamp.events_sorted")

In [121]:
%%sql

SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'sorted' 
FROM demo.bootcamp.events_sorted.files

UNION ALL
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'unsorted' 
FROM demo.bootcamp.events_unsorted.files





size,num_files,sorted
2896920,4,sorted
3211534,4,unsorted


In [90]:
%%sql
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files FROM demo.bootcamp.events.files;

size,num_files
3145713,5


In [ ]:
%%sql 
SELECT COUNT(1) FROM bootcamp.matches_bucketed.files

count(1)
3665
